In [1]:
from can.interface import Bus
from cantools.database import load_file
from broqer import Sink
from canbro import Node
import ipywidgets as widgets
import logging
logging.basicConfig(level=logging.DEBUG)

In [2]:
# load dbc file
db = load_file('poc/device_CAN.dbc')

In [3]:
# create ECU node with virtual bus test
bus_e= Bus('test', interface='virtual')
ecu = Node(name="ECU",bus=bus_e,database=db )

DEBUG:can:can config: {'interface': 'virtual', 'channel': 'test'}
DEBUG:root:Create signal values for <canbro.node.Node object at 0x7f1599d2cfe0>
DEBUG:root:['ECU']
DEBUG:root:Create sender message STATUS
DEBUG:root:create message STATUS
INFO:root:E2E check is not implemented yet for received messages
DEBUG:root:['ECU']
DEBUG:root:Create sender message MEASURE
DEBUG:root:create message MEASURE
INFO:root:E2E check is not implemented yet for received messages
DEBUG:root:['CONTROL']
DEBUG:root:Create receiver message DEM
DEBUG:root:create message DEM
INFO:root:E2E check is not implemented yet for received messages
DEBUG:root:['CONTROL']
DEBUG:root:Create receiver message NM_Control_MSG
DEBUG:root:create message NM_Control_MSG
INFO:root:E2E check is not implemented yet for received messages


In [4]:
# create VCU node with virtual bus test, and connect to ECU via same name of bus -> test
bus_v= Bus('test', interface='virtual')
vcu= Node(name='CONTROL',bus=bus_v,database=db )

DEBUG:can:can config: {'interface': 'virtual', 'channel': 'test'}
DEBUG:root:Create signal values for <canbro.node.Node object at 0x7f15a83d0c50>
DEBUG:root:['ECU']
DEBUG:root:Create receiver message STATUS
DEBUG:root:create message STATUS
INFO:root:E2E check is not implemented yet for received messages
DEBUG:root:['ECU']
DEBUG:root:Create receiver message MEASURE
DEBUG:root:create message MEASURE
INFO:root:E2E check is not implemented yet for received messages
DEBUG:root:['CONTROL']
DEBUG:root:Create sender message DEM
DEBUG:root:create message DEM
INFO:root:E2E check is not implemented yet for received messages
DEBUG:root:['CONTROL']
DEBUG:root:Create sender message NM_Control_MSG
DEBUG:root:create message NM_Control_MSG
INFO:root:E2E check is not implemented yet for received messages


In [5]:
print(vcu.DEM._signal_operation_mode._metadata.initial)

Standby


In [6]:
print(vcu.DEM._signal_DEM_CRC._metadata.initial)

255


In [7]:
vcu.DEM.start_periodic()

DEBUG:root:set initial value of signal DEM_CRC to 255 with type <class 'int'>
DEBUG:root:set initial value of signal DEM_SNC to 1 with type <class 'int'>
DEBUG:root:set initial value of signal operation_mode to Standby with type <class 'cantools.database.namedsignalvalue.NamedSignalValue'>
DEBUG:root:set initial value of signal li_set to 1 with type <class 'int'>
DEBUG:root:set initial value of signal hv_set to 221 with type <class 'int'>
DEBUG:root:set initial value of signal lv_set to 13.0 with type <class 'float'>
DEBUG:root:Update CAN message: DEM
DEBUG:root:Update CAN Data: {'DEM_CRC': 255, 'DEM_SNC': 1, 'operation_mode': 'Standby', 'li_set': 1, 'hv_set': 221, 'lv_set': 13.0}
DEBUG:root:Update CAN message: DEM
DEBUG:root:set periodic publisher for message DEM
DEBUG:root:Update CAN Data: {'DEM_CRC': 255, 'DEM_SNC': 1, 'operation_mode': 'Standby', 'li_set': 1, 'hv_set': 221, 'lv_set': 13.0}


DEBUG:root:Update CAN message: DEM
DEBUG:root:Update CAN Data: {'DEM_CRC': 255, 'DEM_SNC': 1, 'operation_mode': 0, 'li_set': 1, 'hv_set': 221, 'lv_set': 13.0}
DEBUG:root:Update CAN message: DEM
DEBUG:root:Update CAN Data: {'DEM_CRC': 255, 'DEM_SNC': 1, 'operation_mode': 0, 'li_set': 1, 'hv_set': 221, 'lv_set': 13.0}
DEBUG:root:Update CAN message: DEM
DEBUG:root:Update CAN Data: {'DEM_CRC': 255, 'DEM_SNC': 1, 'operation_mode': 0, 'li_set': 1, 'hv_set': 221, 'lv_set': 13.0}
DEBUG:root:Update CAN message: DEM
DEBUG:root:Update CAN Data: {'DEM_CRC': 255, 'DEM_SNC': 1, 'operation_mode': 0, 'li_set': 1, 'hv_set': 221, 'lv_set': 13.0}
DEBUG:root:Update CAN message: DEM
DEBUG:root:Update CAN Data: {'DEM_CRC': 255, 'DEM_SNC': 1, 'operation_mode': 0, 'li_set': 1, 'hv_set': 221, 'lv_set': 13.0}
DEBUG:root:Update CAN message: DEM
DEBUG:root:Update CAN message: DEM
DEBUG:root:Update CAN Data: {'DEM_CRC': 255, 'DEM_SNC': 1, 'operation_mode': 0, 'li_set': 1, 'hv_set': 221, 'lv_set': 13.0}
DEBUG:root:

In [9]:
#vcu.DEM._signal_operation_mode.notify(0)

In [10]:
label_value = widgets.HTML(value="value=?")
def show_vcu_value(value, label: widgets.Label):
    label.value = 'operation mode for ECU node: <br> value={}'.format(value)

show_print = ecu.DEM._signal_operation_mode.subscribe(Sink(show_vcu_value, label=label_value))


In [12]:
list_op_mode = list(set([str(x) for x in list(reversed(vcu.DEM._signal_operation_mode._metadata.conversion.choices.values()))]))

In [13]:
# Check if the value is in the list
if str(vcu.DEM._signal_operation_mode._state) in list_op_mode:
    value = str(vcu.DEM._signal_operation_mode._state)
else:
    value = list_op_mode[0]  # Set a default value

vcu_mode = widgets.ToggleButtons(
    options=list_op_mode,
    description=ecu.DEM._signal_operation_mode._metadata.name + ' selection for VCU node:',
    disabled=False,
    button_style='', 
    tooltips=list_op_mode,
    value=value,
)
# 'success', 'info', 'warning', 'danger' or ''

DEBUG:Comm:handle_msg[22522c830c5a463ab770ca950b91310c]({'header': {'date': datetime.datetime(2024, 3, 4, 17, 5, 37, 532000, tzinfo=tzutc()), 'msg_id': '31aeb748-0832-41c6-bdd3-552d4695bc23', 'msg_type': 'comm_msg', 'session': 'ee95bb82-5f91-4ec0-b5f3-495d17d9af5c', 'username': '94b2a900-119f-4b24-a62b-dfa75b495d2a', 'version': '5.2'}, 'msg_id': '31aeb748-0832-41c6-bdd3-552d4695bc23', 'msg_type': 'comm_msg', 'parent_header': {}, 'metadata': {}, 'content': {'comm_id': '22522c830c5a463ab770ca950b91310c', 'data': {'method': 'update', 'state': {'index': 2}, 'buffer_paths': []}}, 'buffers': []})
DEBUG:Comm:handle_msg[22522c830c5a463ab770ca950b91310c]({'header': {'date': datetime.datetime(2024, 3, 4, 17, 5, 37, 533000, tzinfo=tzutc()), 'msg_id': '6f381ffd-12a7-42a0-934f-50917573366b', 'msg_type': 'comm_msg', 'session': 'ee95bb82-5f91-4ec0-b5f3-495d17d9af5c', 'username': '94b2a900-119f-4b24-a62b-dfa75b495d2a', 'version': '5.2'}, 'msg_id': '6f381ffd-12a7-42a0-934f-50917573366b', 'msg_type': 'c

In [14]:
def set_vcu_mode(value):
    #print('set value to {}'.format(value))
    vcu.DEM._signal_operation_mode.notify(value['new'])
    
vcu_mode.observe(set_vcu_mode, 'value')

In [15]:
def val_vcu_value(value):
    global a_op_mode
    a_op_mode = value

show_print_var = ecu.DEM._signal_operation_mode.subscribe(Sink(val_vcu_value))

In [16]:
display(vcu_mode)
display(label_value)

ToggleButtons(description='operation_mode selection for VCU node:', options=('Boost', 'Pre-charge', 'Active Di…

HTML(value='operation mode for ECU node: <br> value=Standby')

In [17]:
ecu.DEM._signal_operation_mode._state

'Standby'

In [ ]:
ecu.__dict__["DEM"]

In [ ]:
vcu.__dict__["DEM"]